In [1]:
import sqlite3
import pandas as pd

In [2]:
#Make sure all of your BLAST data is concatenated into one file, you can do this easily on the command line with: 
#! cat *.blastp >> all.blastp
#Assuming you are in the correct folder with only blastp files that you want to merge together. 

In [3]:
#Take a look at your data to see what the structure looks like
! head /stor/work/Marcotte/...


D4994_C39_H1_Bin_33_scaffold_102981_1,gb|MBD3186337.1|,0.0,49.109,617,278,9,1,608,1,590,596
D4994_C39_H1_Bin_33_scaffold_102981_1,gb|NMC04130.1|,5.34e-72,33.564,578,348,17,6,565,14,573,253
D4994_C39_H1_Bin_33_scaffold_102981_1,gb|MBN2151694.1|,1.16e-71,33.937,607,340,21,43,607,43,630,252
D4994_C39_H1_Bin_33_scaffold_102981_1,gb|RLI66058.1|,3.00e-17,24.605,569,338,19,55,564,38,574,97.1
D4994_C39_H1_Bin_33_scaffold_102981_1,gb|MBY8979108.1|,1.48e-14,23.234,538,340,18,91,605,88,575,88.6
D4994_C39_H1_Bin_33_scaffold_102981_15,gb|MBD3185777.1|,0.0,67.808,1081,332,5,1,1078,11,1078,1524
D4994_C39_H1_Bin_33_scaffold_102981_15,gb|MBN2150193.1|,0.0,51.288,1359,599,20,1,1322,1,1333,1357
D4994_C39_H1_Bin_33_scaffold_102981_15,gb|NMC04655.1|,0.0,56.422,911,372,10,1,893,1,904,1051
D4994_C39_H1_Bin_33_scaffold_102981_15,gb|MBN2156595.1|,0.0,39.831,1180,636,25,7,1157,4,1138,831
D4994_C39_H1_Bin_33_scaffold_102981_15,gb|TFG17833.1|,0.0,38.549,1227,680,23,7,1206,4,1183,825


In [6]:
#filepath for the sqlite database
clustered_nr_database = "/stor/work/Marcotte/project/drbarth/scratch/clustered_nr/nr_cluster_taxid_formatted_final.sqlite"
#Create a connection to the database
conn = sqlite3.connect(clustered_nr_database)

In [7]:
#read and display the first few rows of the database to get an idea of the format
clustered_nr = pd.read_sql_query("SELECT * FROM nr_cluster_taxid_table LIMIT 1000", conn)
clustered_nr.head()

In [4]:
#Read in the BLAST results that we want to annotate 
blast_files = "YOURFPHERE.blastp"
blast = pd.read_csv(blast_files, sep=',', header=None)
#name the columns
blast.columns = ['qseqid', 'sseqid', 'evalue', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'bitscore']

,qseqid,sseqid,evalue,pident,length,mismatch,gapopen,qstart,qend,sstart,send,bitscore
11525,D4994_C39_H1_Bin_188_scaffold_214728_5,MCF2139298.1,0.000000e+00,66.744,433,133,2,1,432,1,423,587.0
11526,D4994_C39_H1_Bin_188_scaffold_214728_5,UYP45920.1,1.540000e-146,56.546,359,156,0,1,359,1,359,432.0
11527,D4994_C39_H1_Bin_188_scaffold_214728_5,WP_147661481.1,2.480000e-136,56.338,355,143,3,1,351,1,347,407.0
11528,D4994_C39_H1_Bin_188_scaffold_214728_5,UYP45918.1,5.240000e-127,48.698,384,189,2,1,376,1,384,382.0
11529,D4994_C39_H1_Bin_188_scaffold_214728_5,RLI63271.1,4.150000e-101,69.444,216,66,0,1,216,1,216,310.0


In [ ]:
#This may or may not be necessary depending on your formatting, my results the sseqid column had || around the sequence identifier which didn't match the format of the taxonomy database 
#parse the sseqid column to keep all characters inside the | |
blast['sseqid'] = blast['sseqid'].str.extract(r'\|([^|]+)\|')

#check to make sure it worked
blast.head()

In [9]:
#Define the table to query
table_name = "nr_cluster_taxid_table"

#Define the column to query
column_name = "rep"

# make a vector for the sequence ids to retrieve lineages from the sql db for
value_list = blast['sseqid'].tolist()

# Define the query -- create a list of question marks for each value in the value_list
#query = f"SELECT * FROM nr_cluster_taxid_table WHERE rep IN ({','.join(['?']*len(value_list))})"

# Alternative query using tuple
query = f"SELECT * FROM nr_cluster_taxid_table WHERE rep IN {tuple(value_list)}"

# Execute the query and store the result in a new pandas DataFrame
blast_tax_df = pd.read_sql_query(query, conn)#, params=value_list)
#This will take a few minutes to run 

# Close the database connection when you're done
conn.close()



In [10]:
blast_tax_df.head()


,rep,taxid,lca_taxid,lca_lineage_named,lca_lineage_taxid
0,AFV24257.1,1094980,1094980,Archaea;unclassified Archaea kingdom;Euryarcha...,2157;;28890;224756;94695;2206;2220;420950;1094980
1,AGB04797.1,673860,673860,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;2283796;;;;379546;673860;
2,ALM48519.1,96345,96345,Bacteria;unclassified Bacteria kingdom;Bactero...,2;;976;117743;200644;49546;237;96345;
3,APR85540.1,888845,888845,Bacteria;unclassified Bacteria kingdom;Myxococ...,2;;2818505;;3031712;49;1649470;888845;
4,AQS29174.1,115547,115547,Archaea;unclassified Archaea kingdom;unclassif...,2157;;;;;;;115547;


In [11]:
print(f"Number of entries in the blast data: {blast.shape[0]} and number of entries in the found taxonomy data: {blast_tax_df.shape[0]}")

Number of entries in the blast data: 41387 and number of entries in the found taxonomy data: 25207


In [12]:
#Next thing is to merge the blast data with the taxonomic data.
#Use an outer join to merge blast and blast_tax_df on their sseqid and rep columns respectively, to preserve all data from both dataframes
blast_plus_tax = pd.merge(blast, blast_tax_df, how='outer', left_on='sseqid', right_on='rep')
#Drop the rep column
blast_plus_tax = blast_plus_tax.drop('rep', axis=1)
blast_plus_tax.head()

,qseqid,sseqid,evalue,pident,length,mismatch,gapopen,qstart,qend,sstart,send,bitscore,taxid,lca_taxid,lca_lineage_named,lca_lineage_taxid
0,D4994_C39_H1_Bin_33_scaffold_102981_1,MBD3186337.1,0.000000e+00,49.109,617,278,9,1,608,1,590,596.0,2026714,2026714.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;928852;;;;;2026714;
1,D4994_C39_H1_Bin_733_scaffold_586605_5,MBD3186337.1,2.740000e-81,34.828,580,310,20,65,619,51,587,278.0,2026714,2026714.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;928852;;;;;2026714;
2,D4994_C39_H1_Bin_33_scaffold_102981_1,NMC04130.1,5.340000e-72,33.564,578,348,17,6,565,14,573,253.0,2053489,2053489.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;1655434;;;;;2053489;
3,D4994_C39_H1_Bin_733_scaffold_586605_5,NMC04130.1,3.340000e-49,30.092,545,331,19,76,595,66,585,192.0,2053489,2053489.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;1655434;;;;;2053489;
4,D4994_C39_H1_Bin_33_scaffold_102981_1,MBN2151694.1,1.160000e-71,33.937,607,340,21,43,607,43,630,252.0,2053489,2053489.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;1655434;;;;;2053489;


In [13]:
#Show NA values
blast_plus_tax.isna().sum()
#Show the na entries
blast_plus_tax[blast_plus_tax.isna().any(axis=1)]  
blast_plus_tax.head()

,qseqid,sseqid,evalue,pident,length,mismatch,gapopen,qstart,qend,sstart,send,bitscore,taxid,lca_taxid,lca_lineage_named,lca_lineage_taxid
0,D4994_C39_H1_Bin_33_scaffold_102981_1,MBD3186337.1,0.000000e+00,49.109,617,278,9,1,608,1,590,596.0,2026714,2026714.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;928852;;;;;2026714;
1,D4994_C39_H1_Bin_733_scaffold_586605_5,MBD3186337.1,2.740000e-81,34.828,580,310,20,65,619,51,587,278.0,2026714,2026714.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;928852;;;;;2026714;
2,D4994_C39_H1_Bin_33_scaffold_102981_1,NMC04130.1,5.340000e-72,33.564,578,348,17,6,565,14,573,253.0,2053489,2053489.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;1655434;;;;;2053489;
3,D4994_C39_H1_Bin_733_scaffold_586605_5,NMC04130.1,3.340000e-49,30.092,545,331,19,76,595,66,585,192.0,2053489,2053489.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;1655434;;;;;2053489;
4,D4994_C39_H1_Bin_33_scaffold_102981_1,MBN2151694.1,1.160000e-71,33.937,607,340,21,43,607,43,630,252.0,2053489,2053489.0,Archaea;unclassified Archaea kingdom;Candidatu...,2157;;1655434;;;;;2053489;


In [14]:
#Further formatting if desired
#Group blast_plus_tax by qseqid and combine all hits into a list
blast_plus_tax_grouped = blast_plus_tax.groupby('qseqid').agg({
    'sseqid': list,
    'evalue': list,
    'taxid': list,
    'lca_taxid': list,
    'lca_lineage_named': list,
    'lca_lineage_taxid': list
    }).reset_index()

blast_plus_tax_grouped.head()
#blast_plus_tax_grouped.shape

,qseqid,sseqid,evalue,taxid,lca_taxid,lca_lineage_named,lca_lineage_taxid
0,D4994_C39_H1_Bin_122_scaffold_100775_1,"[MCF2139333.1, WP_162306510.1, UYP45893.1, RLI...","[1.96e-76, 1.49e-76, 2.02e-82, 2.66e-73, 1.37e...","[2053489, 2594042, 2951803, 2053489, 2026747]","[2053489.0, 2594042.0, 2951803.0, 2053489.0, 2...",[Archaea;unclassified Archaea kingdom;Candidat...,"[2157;;1655434;;;;;2053489;, 2157;;1655434;;;;..."
1,D4994_C39_H1_Bin_122_scaffold_100775_2,"[RLI64734.1, MCF2139334.1, UYP45892.1, WP_1623...","[1.72e-106, 1.22e-112, 1.07e-110, 5.48e-91, 4....","[2053489, 2053489, 2951803, 2594042, 2053489]","[2053489.0, 2053489.0, 2951803.0, 2594042.0, 2...",[Archaea;unclassified Archaea kingdom;Candidat...,"[2157;;1655434;;;;;2053489;, 2157;;1655434;;;;..."
2,D4994_C39_H1_Bin_122_scaffold_110030_13,"[TES98119.1, MBN1803189.1, OGN89944.1, UYP4859...","[1.92e-116, 1.07e-110, 1.43e-109, 1.32e-149, 9...","[2053489, 2053489, 1797620, 2951803, 2026747]","[2053489.0, 2053489.0, 1797620.0, 2951803.0, 2...",[Archaea;unclassified Archaea kingdom;Candidat...,"[2157;;1655434;;;;;2053489;, 2157;;1655434;;;;..."
3,D4994_C39_H1_Bin_122_scaffold_110030_15,"[MCF2139795.1, UYP45185.1, WP_147664550.1, MCK...","[7.3e-10, 2.47e-15, 2.24e-14, 1.61e-09, 1.99e-14]","[2053489, 2951803, 2594042, 2053489, 2951803]","[2053489.0, 2951803.0, 2594042.0, 2053489.0, 2...",[Archaea;unclassified Archaea kingdom;Candidat...,"[2157;;1655434;;;;;2053489;, 2157;;1655434;;;;..."
4,D4994_C39_H1_Bin_122_scaffold_110030_6,"[NVM29962.1, NVM55174.1, NHI91303.1, MBY900747...","[6.39e-171, 3.71e-166, 3.76e-160, 7.13e-78, 1....","[2719382, 2719382, 2053489, 2053489, 2053489]","[2719382.0, 2719382.0, 2053489.0, 2053489.0, 2...",[Archaea;unclassified Archaea kingdom;Candidat...,"[2157;;2572044;;;;;2719382;, 2157;;2572044;;;;..."


In [17]:
! pwd

/stor/work/Marcotte/project/drbarth/asgard


In [18]:
#Export annotated clustered_nr results to a csv file
new_fp = '/stor/work/Marcotte/project/...csv'
blast_plus_tax_grouped.to_csv(new_fp, index=False)